https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b

In [ ]:
import pandas as pd
from math import floor
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

In [2]:
data1 = pd.read_csv('premier_14_15.csv')
data2 = pd.read_csv('premier_15_16.csv')
data3 = pd.read_csv('premier_16_17.csv')
data4 = pd.read_csv('premier_17_18.csv')

dataCon = [data1,data2,data3,data4]
data = pd.concat(dataCon)

In [3]:
b = np.random.uniform(low=-0.01,high=0.01,size=(3,3))
a = pd.DataFrame(b,columns=['a','b','c'])
a

,a,b,c
0,-0.008541,0.000080,-0.005617
1,0.001229,-0.001305,-0.005408
2,-0.008288,-0.000809,-0.005732


In [4]:
def normalization(raw_data):
    for col_num in range(raw_data.shape[1]):
        if raw_data.iloc[:,col_num].dtype == np.float or raw_data.iloc[:,col_num].dtype == np.int:
            raw_data.iloc[:,col_num] = (raw_data.iloc[:,col_num] - raw_data.iloc[:,col_num].mean()) / (raw_data.iloc[:,col_num].max() - raw_data.iloc[:,col_num].min())
    return raw_data

In [5]:
def embedding_matrix(column):
    labels = []
    embeddings = np.array([])
    num_of_uniques = len(np.unique(column))
    for i in range(num_of_uniques):
        if embeddings.size == 0:
            embeddings = np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1))
        else:
            embeddings = np.append(embeddings,np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1)),axis=1)
        labels.append(np.unique(column)[i])
    print("embeddings.shape:",embeddings.shape)
    print("labels.shape:",len(labels))
    return pd.DataFrame(data=embeddings,columns=labels)

In [6]:
em = embedding_matrix(data['HomeTeam'])
em.head()

embeddings.shape: (13, 26)
labels.shape: 26


,Arsenal,Aston Villa,Bournemouth,Brighton,Burnley,Chelsea,Crystal Palace,Everton,Huddersfield,Hull,...,Norwich,QPR,Southampton,Stoke,Sunderland,Swansea,Tottenham,Watford,West Brom,West Ham
0,0.001590,-0.004891,0.003323,0.007017,-0.001090,0.009396,0.005551,0.000422,0.002859,0.008952,...,0.009745,0.002418,0.004941,0.000185,-0.006953,-0.008540,-0.006345,0.007370,-0.005609,-0.003167
1,0.005887,0.004213,0.000498,-0.002223,0.004186,-0.007770,0.006134,-0.006477,0.005590,0.006993,...,0.003250,-0.008823,-0.006330,-0.001342,-0.002282,0.001700,0.006963,0.005683,-0.001092,0.007153
2,-0.009216,0.006366,-0.008589,-0.007370,-0.009598,0.003301,-0.002124,-0.003433,-0.008858,0.006337,...,-0.005757,0.007096,-0.008579,0.002129,-0.006301,-0.003821,0.005474,0.005867,-0.008017,-0.000285
3,-0.007318,0.007023,0.002411,0.008953,0.002512,0.001036,-0.006038,-0.001937,-0.001351,0.006068,...,-0.006099,-0.005628,-0.003683,0.001746,0.004723,0.006565,0.007207,-0.007662,-0.002428,-0.002846
4,0.006347,0.007581,-0.008103,-0.001990,-0.002739,-0.008141,0.000023,-0.001402,-0.009501,-0.005459,...,-0.007385,-0.005175,-0.004032,-0.001858,0.002595,-0.007370,-0.008091,-0.006911,0.005912,-0.003664


In [9]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
data = normalization(data)
x_data = np.column_stack((np.transpose(em[data['HomeTeam']].values),
                          np.transpose(em[data['AwayTeam']].values),
                          data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [8]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
x_data = np.column_stack((data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [10]:
x_data.shape

(1520, 29)

In [11]:
# Two first games
x_data[:,0:2]

array([[ 0.00159047,  0.00588698],
       [ 0.00127791, -0.00777876],
       [-0.00735872, -0.00749324],
       ...,
       [-0.00853983,  0.00170008],
       [-0.00634513,  0.00696253],
       [-0.00316654,  0.00715321]])

In [12]:
y_data.shape

(1520, 3)

In [13]:
y_data.head()

,A,D,H
0,0,0,1
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0


In [14]:
train_size = 0.9
valid_size = 0.3

train_cnt = floor(x_data.shape[0] * train_size)

x_train = x_data[0:train_cnt]
y_train = y_data.iloc[0:train_cnt].values

valid_cnt = floor((x_data.shape[0] - train_cnt) * valid_size)

x_valid = x_data[train_cnt:train_cnt+valid_cnt]
y_valid = y_data.iloc[train_cnt:train_cnt+valid_cnt].values

x_test = x_data[train_cnt+valid_cnt:]
y_test = y_data.iloc[train_cnt+valid_cnt:]

print("x_train:",x_train.shape)
print("x_valid:",x_valid.shape)
print("x_test:",x_test.shape)

x_train: (1368, 29)
x_valid: (45, 29)
x_test: (107, 29)


In [15]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [16]:
# Parameters
ALPHA = 1e-3
num_epochs = 50
batch_size = 128
display_step = 5

In [24]:
# Network Parameters
num_input = x_data.shape[1]
num_classes = y_data.shape[1]
num_hidden_1 = 50
num_hidden_2 = 50
KEEP_PROBABILITY = 0.9

In [25]:
def neural_network(x,weights,biases,keep_prob):
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    layer_out = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return layer_out

In [26]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([num_input,num_hidden_1])),
    'w2': tf.Variable(tf.random_normal([num_hidden_1,num_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

keep_prob = tf.placeholder("float")

In [27]:
predictions = neural_network(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=ALPHA).minimize(cost)

In [28]:
costs = []
lrn_rate_sizes = []

In [36]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: KEEP_PROBABILITY
                            })
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Train: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(avg_cost))
            _, valid_cost = sess.run([optimizer, cost], feed_dict={x: x_valid, y: y_valid, keep_prob: 1})
            print("Valid: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(valid_cost))
            
            
            prediction=tf.argmax(predictions,1)
            print("Predictions:",prediction.eval(feed_dict={x: x_test, y: y_test, keep_prob: 1.0})[:10])
            
            
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
            print("______________________________________")
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Train: Epoch: 0005 cost= 23.658637047
Valid: Epoch: 0005 cost= 14.660253525
Predictions: [2 2 2 2 2 2 2 2 2 2]
Accuracy: 0.45794392
______________________________________
Train: Epoch: 0010 cost= 10.127031946
Valid: Epoch: 0010 cost= 3.669965506
Predictions: [2 2 2 2 2 2 2 2 2 2]
Accuracy: 0.47663552
______________________________________
Train: Epoch: 0015 cost= 5.159888792
Valid: Epoch: 0015 cost= 3.029505491
Predictions: [2 2 1 2 0 2 0 2 2 2]
Accuracy: 0.5233645
______________________________________
Train: Epoch: 0020 cost= 4.547799873
Valid: Epoch: 0020 cost= 2.352024555
Predictions: [2 2 2 2 0 2 0 2 2 2]
Accuracy: 0.55140185
______________________________________
Train: Epoch: 0025 cost= 4.034049439
Valid: Epoch: 0025 cost= 2.002253294
Predictions: [2 2 1 2 0 2 0 2 2 2]
Accuracy: 0.49532712
______________________________________
Train: Epoch: 0030 cost= 3.919442439
Valid: Epoch: 0030 cost= 1.969116926
Predictions: [2 2 0 2 0 0 0 1 2 2]
Accuracy: 0.48598132
_______________________